In [42]:
import pandas as pd
import altair as alt
import eco_style
alt.themes.enable('light')
import numpy as np
import statsmodels.api as sm


# Scot Polling

In [35]:
df

,date,party,value,days,color
0,2020-03-26 03:15:00,Conservative,0.27,0,#0087DC
1,2020-04-27 03:13:20,Conservative,0.25,31,#0087DC
2,2020-05-05 03:10:00,Conservative,0.26,39,#0087DC
3,2020-06-05 03:11:40,Conservative,0.21,70,#0087DC
4,2020-07-03 03:08:20,Conservative,0.21,98,#0087DC
...,...,...,...,...,...
585,2024-06-02 00:06:40,Other,0.09,1528,#666666
586,2024-06-09 00:03:20,Other,0.08,1535,#666666
587,2024-06-10 00:05:00,Other,0.09,1536,#666666
588,2024-06-14 00:01:40,Other,0.13,1540,#666666


In [38]:
dt1 = pd.to_datetime("2020-03-26 03:15:00")
dt2 = pd.to_datetime("2024-06-18 00:00:00")
dt2 - dt1

dt1 

Timedelta('1544 days 20:45:00')

In [34]:
(df['date'] - df['date'].min())

0        0 days 00:00:00
1       31 days 23:58:20
2       39 days 23:55:00
3       70 days 23:56:40
4       98 days 23:53:20
             ...        
585   1528 days 20:51:40
586   1535 days 20:48:20
587   1536 days 20:50:00
588   1540 days 20:46:40
589   1544 days 20:45:00
Name: date, Length: 590, dtype: timedelta64[ns]

In [68]:
df = pd.read_csv("WhatScotlandThinks_UK_data.csv")
df['Category'] = pd.to_datetime(df['Category'], unit='ms')
df = df.rename(columns={'Category': 'date'}).drop(columns=['Navigator 1'])
df = df.melt(id_vars=['date'], var_name='party', value_name='value')
df['days'] = (df['date'] - df['date'].min()).dt.days
df['value'] = df['value']/100

#do a loess with np

x_vals = list(range(0, df['days'].max()))
loess_dfs = []
for party in df['party'].unique():
    loess = sm.nonparametric.lowess(df[df['party'] == party]['value'], df[df['party'] == party]['days'], frac=0.15)
    loess = pd.DataFrame(loess, columns=['days', 'value'])
    loess['party'] = party
    loess_dfs.append(loess)

loess_df = pd.concat(loess_dfs)
loess_df['date'] = df['date'].min() + pd.to_timedelta(loess_df['days'], unit='D')
loess_df['label'] = np.where(loess_df['days'] == 1544.0, loess_df['party'], '')



In [83]:


party_colors = {
    'Conservative': '#0087DC',
    'Labour': '#DC241f',
    'Liberal Democrat': '#FAA61A',
    'Scottish National Party': 'rgb(210, 169, 21)',
    'Other': '#666666'
}

df['color'] = df['party'].map(party_colors)
loess_df['color'] = loess_df['party'].map(party_colors)


base = alt.Chart(df).encode(
    x=alt.X('date:T', title='', axis=alt.Axis(format='%b %Y')),
    y=alt.Y('value:Q', title='', axis=alt.Axis(format='%')),
    color=alt.Color('color', scale=None),
)

points = base.mark_circle(opacity=0.3).encode(
    tooltip=['date:T', 'value:Q', 'party:N']
)

loess = alt.Chart(loess_df).encode(
    x=alt.X('date:T', title='', axis=alt.Axis()),
    y=alt.Y('value:Q', title='', axis=alt.Axis(format='%')),
    color=alt.Color('color:N', scale=None),
).mark_line()

labels = loess.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=alt.expr('datum.party == "Other" ? 10 : 0'),
    fontSize=10,
    color='black',
).encode(
    text='label:N'
)

chart = points + loess + labels

chart = chart.properties(width=400, height=300) 


chart.save('ge.json')
chart.save('ge.png', scale_factor=2.0)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

# Indie Ref

In [95]:
df = pd.read_csv("WhatScotlandThinks_indie_ref.csv")
df = df.rename(columns={'Category': 'date'}).drop(columns=['Navigator 1'])
df['date'] = pd.to_datetime(df['date'], unit='ms')
df = df.melt(id_vars=['date'], var_name='party', value_name='value')
df['days'] = (df['date'] - df['date'].min()).dt.days
df['value'] = df['value']/100

#do a loess with np

x_vals = list(range(0, df['days'].max()))
loess_dfs = []
for party in df['party'].unique():
    loess = sm.nonparametric.lowess(df[df['party'] == party]['value'], df[df['party'] == party]['days'], frac=0.05)
    loess = pd.DataFrame(loess, columns=['days', 'value'])
    loess['party'] = party
    loess_dfs.append(loess)

loess_df = pd.concat(loess_dfs)
loess_df['date'] = df['date'].min() + pd.to_timedelta(loess_df['days'], unit='D')
loess_df['label'] = np.where(loess_df['days'] == 2914.0, loess_df['party'], '')

loess_df


,days,value,party,date,label
0,0.0,0.523625,Yes,2016-06-25 05:03:20,
1,1.0,0.522879,Yes,2016-06-26 05:03:20,
2,3.0,0.521388,Yes,2016-06-28 05:03:20,
3,30.0,0.501447,Yes,2016-07-25 05:03:20,
4,67.0,0.473778,Yes,2016-08-31 05:03:20,
...,...,...,...,...,...
228,2893.0,0.521491,No,2024-05-27 05:03:20,
229,2898.0,0.517925,No,2024-06-01 05:03:20,
230,2905.0,0.512415,No,2024-06-08 05:03:20,
231,2910.0,0.508042,No,2024-06-13 05:03:20,


In [98]:
party_colors = {
    'Yes': '#36B7B4',
    'No': '#E6224B',
}

df['color'] = df['party'].map(party_colors)
loess_df['color'] = loess_df['party'].map(party_colors)


base = alt.Chart(df).encode(
    x=alt.X('date:T', title='', axis=alt.Axis(format='%b %Y')),
    y=alt.Y('value:Q', title='', axis=alt.Axis(format='%')),
    color=alt.Color('color', scale=None),
)

points = base.mark_circle(opacity=0.3).encode(
    tooltip=['date:T', 'value:Q', 'party:N']
)

loess = alt.Chart(loess_df).encode(
    x=alt.X('date:T', title='', axis=alt.Axis()),
    y=alt.Y('value:Q', title='', axis=alt.Axis(format='%')),
    color=alt.Color('color:N', scale=None),
).mark_line()

labels = loess.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=alt.expr('datum.party == "Yes" ? 5 : 0'),
    fontSize=10,
    color='black',
).encode(
    text='label:N'
)

chart = points + loess + labels

chart = chart.properties(width=400, height=300) 


chart.save('indie.json')
chart.save('indie.png', scale_factor=2.0)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [61]:
alt.Chart(loess_df).encode(
    x=alt.X('date:T', title='', axis=alt.Axis()),
    y=alt.Y('value:Q', title='', axis=alt.Axis(format='%')),
    color=alt.Color('party:N', scale=None),
).mark_point().save("foo.json")

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [62]:
loess_df

,days,value,party,date,color
0,0.0,0.255952,Conservative,2020-03-26 03:15:00,#0087DC
1,31.0,0.245347,Conservative,2020-04-26 03:15:00,#0087DC
2,39.0,0.242695,Conservative,2020-05-04 03:15:00,#0087DC
3,70.0,0.232720,Conservative,2020-06-04 03:15:00,#0087DC
4,98.0,0.224001,Conservative,2020-07-02 03:15:00,#0087DC
...,...,...,...,...,...
113,1528.0,0.081851,Other,2024-06-01 03:15:00,#666666
114,1535.0,0.078612,Other,2024-06-08 03:15:00,#666666
115,1536.0,0.078155,Other,2024-06-09 03:15:00,#666666
116,1540.0,0.076348,Other,2024-06-13 03:15:00,#666666


In [56]:
loess_df

,days,value,party,date,color
0,0.0,0.255952,Conservative,2020-03-26 03:15:00,#0087DC
1,31.0,0.245347,Conservative,2020-04-26 03:15:00,#0087DC
2,39.0,0.242695,Conservative,2020-05-04 03:15:00,#0087DC
3,70.0,0.232720,Conservative,2020-06-04 03:15:00,#0087DC
4,98.0,0.224001,Conservative,2020-07-02 03:15:00,#0087DC
...,...,...,...,...,...
113,1528.0,0.081851,Other,2024-06-01 03:15:00,#666666
114,1535.0,0.078612,Other,2024-06-08 03:15:00,#666666
115,1536.0,0.078155,Other,2024-06-09 03:15:00,#666666
116,1540.0,0.076348,Other,2024-06-13 03:15:00,#666666


In [55]:
loess

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [13]:
df = pd.read_csv("WhatScotlandThinks_UK_data.csv")
df['Category'] = pd.to_datetime(df['Category'], unit='ms')
df = df.rename(columns={'Category': 'date'}).drop(columns=['Navigator 1'])
df = df.melt(id_vars=['date'], var_name='party', value_name='value')
df['days'] = (df['date'] - df['date'].min()).dt.days


#loes it
alt.Chart(df).transform_loess(
    'days', 'value', groupby=['party']
).mark_line().encode(
    x='date:T',
    y='value:Q',
    color='party:N'
).properties(width=500, height=300)



/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [37]:
loess

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [31]:
loess = alt.Chart(df).transform_loess(
    on='date',
    loess='value',
    groupby=['party'],
    bandwidth=0.3
).mark_line().encode(
    x='date:T',
    y='value:Q',
    color='color',
)

loess

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [32]:
df

,date,party,value,color
0,2020-03-26 03:15:00,Conservative,0.27,#0087DC
1,2020-04-27 03:13:20,Conservative,0.25,#0087DC
2,2020-05-05 03:10:00,Conservative,0.26,#0087DC
3,2020-06-05 03:11:40,Conservative,0.21,#0087DC
4,2020-07-03 03:08:20,Conservative,0.21,#0087DC
...,...,...,...,...
585,2024-06-02 00:06:40,Other,0.09,#666666
586,2024-06-09 00:03:20,Other,0.08,#666666
587,2024-06-10 00:05:00,Other,0.09,#666666
588,2024-06-14 00:01:40,Other,0.13,#666666


In [17]:
df.party.unique()

array(['Conservative', 'Labour', 'Liberal Democrat',
       'Scottish National Party', 'Other'], dtype=object)